In [1]:
pip install catboost

In [2]:
pip install optuna

In [3]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from sklearn.model_selection import KFold,train_test_split
from sklearn.metrics import roc_auc_score
from catboost import CatBoostClassifier
import optuna

In [13]:
train = pd.read_csv('preprocessed_train.csv', encoding='euc-kr')
test = pd.read_csv('preprocessed_test.csv', encoding='euc-kr')

In [14]:
train.head()

,분석데이터,label,numstrings,avlength,printables,entropy,paths,urls,registry,MZ,a_0,a_1,a_2,a_3,a_4,a_5,a_6,a_7,a_8,a_9,a_10,a_11,a_12,a_13,a_14,a_15,a_16,a_17,a_18,a_19,a_20,a_21,a_22,a_23,a_24,a_25,a_26,a_27,a_28,a_29,...,dist_56,dist_57,dist_58,dist_59,dist_60,dist_61,dist_62,dist_63,dist_64,dist_65,dist_66,dist_67,dist_68,dist_69,dist_70,dist_71,dist_72,dist_73,dist_74,dist_75,dist_76,dist_77,dist_78,dist_79,dist_80,dist_81,dist_82,dist_83,dist_84,dist_85,dist_86,dist_87,dist_88,dist_89,dist_90,dist_91,dist_92,dist_93,dist_94,dist_95
0,1,1,144,12.298611,1771,5.356616,0,0,0,1,2399,572,519,673,555,516,558,510,709,640,515,854,571,675,510,511,508,485,423,432,387,393,468,428,413,382,377,422,374,366,...,1,20,0,20,2,22,22,23,1,79,9,35,37,158,14,22,21,89,9,2,65,26,96,89,31,0,106,42,127,40,10,4,10,9,4,0,1,0,0,0
1,2,1,804,9.580846,7703,6.063542,0,0,0,6,183376,8477,4514,3921,7260,2869,2414,2140,8056,1999,1721,1451,5701,1734,1195,7756,5249,1699,1301,1186,2933,2779,1050,693,2339,948,658,736,1488,595,...,27,51,21,21,33,30,19,38,23,214,55,113,153,538,46,70,57,230,29,40,263,118,196,224,85,33,244,184,332,113,43,121,84,78,47,36,40,45,27,36
2,3,0,2205,12.736054,28083,6.107050,9,0,0,6,1178,28,9,6,20,4,6,0,2,2,4,3,1,3,4,0,19,3,1,0,1,0,0,0,0,0,0,1,2,1,...,417,217,127,719,342,372,65,148,85,722,172,551,416,1670,192,284,233,912,169,297,603,588,743,791,387,196,841,695,1072,381,326,268,239,286,199,148,154,37,48,36
3,4,0,2602,10.288240,26770,5.373013,8,0,0,1,56851,2625,1520,1351,3266,802,1173,681,3957,1187,719,363,2765,764,429,1558,2081,647,242,292,1402,1634,265,275,1013,381,256,255,759,233,...,46,1,37,1,55,1,0,562,36,1381,693,236,515,2540,306,281,921,864,740,119,1022,379,1245,881,326,7,1153,836,1486,479,336,230,206,245,76,0,26,702,1,5
4,5,1,8980,23.252339,208806,5.775223,0,28,16,3,124274,5778,3669,3595,6210,11804,12076,10340,5722,1894,1537,1599,3961,2104,1473,5067,4205,1760,1431,1707,2784,2282,1915,1415,2239,2190,1665,1454,2098,1744,...,2351,221,2575,225,578,265,276,2291,357,6689,3771,2731,3886,11818,1213,1966,2089,10984,329,677,4487,2189,6430,6891,2175,163,5861,4166,10355,5049,731,882,1171,1010,322,64,327,84,75,244


Building Model - CatBoost

In [15]:
y = train['label']
train.drop('label',axis=1,inplace=True)

In [16]:
train.drop('분석데이터',axis=1,inplace=True)
test.drop('분석대상',axis=1,inplace=True)

Optuna

In [17]:
def fit_cat(trial, x_train, y_train, x_test, y_test):
    params = {'iterations':trial.suggest_int("iterations", 1000, 100000),
              'od_wait':trial.suggest_int('od_wait', 500, 5000),
              'task_type':"GPU",
              'learning_rate' : trial.suggest_uniform('learning_rate', 0.02 , 0.06),
              'reg_lambda': trial.suggest_loguniform('reg_lambda', 0.30 , 0.33),
              'subsample': trial.suggest_uniform('subsample',0.8,1.0),
              'random_strength': trial.suggest_uniform('random_strength',10,50),
              'depth': trial.suggest_int('depth',1,15),
              'min_data_in_leaf': trial.suggest_int('min_data_in_leaf',1,50),
              'leaf_estimation_iterations': trial.suggest_int('leaf_estimation_iterations',1,15),
              'bootstrap_type':'Poisson'
               }
    
    
    model = CatBoostClassifier(**params)
    model.fit(x_train, y_train,eval_set=[(x_test,y_test)], early_stopping_rounds=150, verbose=False)
    
    y_train_pred = model.predict_proba(x_train)[:,1]
    
    y_test_pred = model.predict_proba(x_test)[:,1]
    y_train_pred = np.clip(y_train_pred, 0.1, None)
    y_test_pred = np.clip(y_test_pred, 0.1, None)
    
    log = {
        "train roc_auc": roc_auc_score(y_train, y_train_pred),
        "valid roc_auc": roc_auc_score(y_test, y_test_pred)
    }
    
    return model, log

catboost

In [18]:
# catboost params
cat_params = {'iterations': 2866,
 'od_wait': 3385,
 'learning_rate': 0.04280810491488757,
 'reg_lambda': 0.32139709692279206,
 'subsample': 0.8442605943226449,
 'random_strength': 22.468752639603235,
 'depth': 4,
 'min_data_in_leaf': 31,
 'leaf_estimation_iterations': 15,
 'task_type':"GPU",
 'bootstrap_type':'Poisson'}

In [19]:
folds = KFold(n_splits = 5, random_state = 2021, shuffle = True)

predictions = np.zeros(len(test))

for fold, (trn_idx, val_idx) in enumerate(folds.split(train)):
    print(f"Fold: {fold}")
    X_train, X_test = train.iloc[trn_idx], train.iloc[val_idx]
    y_train, y_test = y.iloc[trn_idx], y.iloc[val_idx]

    model = CatBoostClassifier(**cat_params)
   
    model.fit(X_train, y_train,
              eval_set=[(X_test, y_test)],
                early_stopping_rounds=400,
                verbose=False)
    pred = model.predict_proba(X_test)[:,1]
    roc = roc_auc_score(y_test, pred)
    print(f" roc_auc_score: {roc}")
    print("-"*50)
    
    predictions += model.predict_proba(test)[:,1] / folds.n_splits 

Fold: 0
 roc_auc_score: 0.9651785795504507
--------------------------------------------------
Fold: 1
 roc_auc_score: 0.9767727272727273
--------------------------------------------------
Fold: 2
 roc_auc_score: 0.9655035830936479
--------------------------------------------------
Fold: 3
 roc_auc_score: 0.9739405351372714
--------------------------------------------------
Fold: 4
 roc_auc_score: 0.9690483368505678
--------------------------------------------------


In [20]:
predictions

array([0.79210585, 0.82458346, 0.93768733, ..., 0.07100135, 0.75799599,
       0.58620343])